In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from joblib import dump, load
from collections import Counter

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# -----------------
# Config
# -----------------
RS = 1234
DATA   = Path("Data/master.parquet")
MODELS = Path("Models"); MODELS.mkdir(exist_ok=True)
PLOTS  = Path("Plots");  PLOTS.mkdir(exist_ok=True)
PLOTS_SVM = PLOTS / "SVM"; PLOTS_SVM.mkdir(parents=True, exist_ok=True)

# -----------------
# IO
# -----------------
def load_xy(drop_cols=("family","genus","species")):
    df = pd.read_parquet(DATA)
    X = df.drop(columns=list(drop_cols), errors="ignore").to_numpy(dtype=np.float32)
    return df, X

# -----------------
# Rarity labels (unsupervised) for training the SVM gate’s target
# -----------------
def initial_rare_labels(species, method="kmeans", random_state=RS):
    cnt = Counter(species)
    counts = np.fromiter((cnt[s] for s in species), dtype=np.int32, count=len(species))
    x = np.log1p(counts).reshape(-1, 1)

    if method == "gmm":
        model = GaussianMixture(n_components=2, random_state=random_state).fit(x)
        rare_comp = np.argmin(model.means_.ravel())
        hard = (model.predict(x) == rare_comp).astype(np.int32)
        soft = model.predict_proba(x)[:, rare_comp]

    elif method == "kmeans":
        model = KMeans(n_clusters=2, n_init=10, random_state=random_state).fit(x)
        rare_comp = np.argmin(model.cluster_centers_.ravel())
        hard = (model.labels_ == rare_comp).astype(np.int32)
        d = np.linalg.norm(x - model.cluster_centers_[rare_comp], axis=1)
        soft = (d.max() - d) / (d.max() - d.min() + 1e-9)

    elif method == "knn":
        k = 10
        model = KNeighborsRegressor(n_neighbors=k).fit(x, counts)
        local_density = model.predict(x)
        soft = 1 - (local_density - local_density.min()) / (local_density.max() - local_density.min() + 1e-9)
        hard = (soft > np.median(soft)).astype(np.int32)

    else:
        raise ValueError("method must be 'gmm','kmeans','knn'")

    return hard, soft

# -----------------
# Simple SVM factories
# -----------------
def SVMLIN():
    clf = Pipeline([
        ("mm", MinMaxScaler()),
        ("svc", SVC(kernel="linear", probability=True, tol=1e-2, max_iter=1500,
                    decision_function_shape="ovr", random_state=RS))
    ])
    clf.name = "SVMLIN"
    return clf

def SVMRBF():
    clf = Pipeline([
        ("mm", MinMaxScaler()),
        ("svc", SVC(kernel="rbf", probability=True, tol=1e-2, max_iter=1500,
                    decision_function_shape="ovr", random_state=RS))
    ])
    clf.name = "SVMRBF"
    return clf

def SVMPoly(d=2):
    clf = Pipeline([
        ("mm", MinMaxScaler()),
        ("svc", SVC(kernel="poly", degree=d, probability=True, tol=1e-2, max_iter=1500,
                    decision_function_shape="ovr", random_state=RS))
    ])
    clf.name = f"SVMPoly{d}"
    return clf

In [2]:
def stage1_train(train_size=0.75, methods=("gmm","kmeans","knn")):
    df, X = load_xy()
    species = df["species"].to_numpy()

    factories = [SVMLIN, SVMRBF, lambda: SVMPoly(2), lambda: SVMPoly(3), lambda: SVMPoly(4), lambda: SVMPoly(5)]

    best = None
    for method in methods:
        y_rare, _ = initial_rare_labels(species, method=method)
        Xtr, Xte, ytr, yte = train_test_split(X, y_rare, test_size=(1-train_size), stratify=y_rare, random_state=RS)

        for factory in factories:
            clf = factory()
            name = getattr(clf, "name", clf.__class__.__name__)
            clf.fit(Xtr, ytr)
            proba = clf.predict_proba(Xte)[:, 1]

            fpr, tpr, thr = roc_curve(yte, proba)
            auc = roc_auc_score(yte, proba)
            youden = tpr - fpr
            t_opt = thr[np.argmax(youden)]

            # save model + meta
            model_dir = MODELS / "SVM"; model_dir.mkdir(exist_ok=True, parents=True)
            tag = f"{method}_{name}_{int(train_size*100)}"
            dump(clf, model_dir / f"{tag}.joblib")
            with open(model_dir / f"{tag}_meta.json", "w", encoding="utf-8") as f:
                json.dump({"method": method, "auc": float(auc), "threshold": float(t_opt)}, f, indent=2)

            # TPR/FPR threshold plot
            fig, ax = plt.subplots(figsize=(8,5))
            ax.plot(thr, tpr, label="TPR", color="blue")
            ax.plot(thr, fpr, label="FPR", color="red")
            ax.plot(thr, (tpr - fpr) - 1, label="TPR−FPR (−1 offset)", color="green")
            ax.invert_xaxis(); ax.set_xlabel("Threshold"); ax.set_ylabel("Rate"); ax.legend(frameon=False); ax.grid(alpha=0.3)
            fig.savefig(PLOTS_SVM / f"{tag}_tprfpr.png", dpi=300, bbox_inches="tight")
            plt.close(fig)

            if (best is None) or (auc > best["auc"]):
                best = {"tag": tag, "method": method, "auc": float(auc), "threshold": float(t_opt)}

    # persist best marker
    with open(MODELS / "SVM" / "_best.json", "w", encoding="utf-8") as f:
        json.dump(best, f, indent=2)
    print(f"[Stage1] Best rarity gate: {best['tag']} (AUC={best['auc']:.3f}, thr={best['threshold']:.3f})")
    return best

In [3]:
# -----------------
# Stage 1 OOF rarity probs for full dataset (no leakage for Stage 3)
# -----------------
def stage1_make_oof_prare(best):
    df, X = load_xy()
    species = df["species"].to_numpy()
    y_rare, _ = initial_rare_labels(species, method=best["method"])

    # Recreate the *same* classifier type from best tag
    tag = best["tag"]  # e.g., "kmeans_SVMRBF_75"
    name = tag.split("_")[1]
    if name == "SVMLIN":
        clf = SVMLIN()
    elif name == "SVMRBF":
        clf = SVMRBF()
    elif name.startswith("SVMPoly"):
        deg = int(name.replace("SVMPoly", ""))
        clf = SVMPoly(deg)
    else:
        raise RuntimeError("Unknown SVM name in tag")

    # OOF probabilities
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RS)
    p_rare_oof = cross_val_predict(clf, X, y_rare, cv=cv, method="predict_proba", n_jobs=1)[:, 1]

    # Save array for reuse
    np.save(MODELS / "SVM" / "p_rare_oof.npy", p_rare_oof)
    return p_rare_oof

In [4]:
# -----------------
# Stage 2: Train Family/Genus models and compute OOF probs for Stage 3
# -----------------
def stage2_train():
    df, X = load_xy()
    yF = df["family"].to_numpy()
    yG = df["genus"].to_numpy()

    fam = RandomForestClassifier(n_estimators=400, n_jobs=-1, random_state=RS)
    gen = RandomForestClassifier(n_estimators=400, n_jobs=-1, random_state=RS)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RS)
    fam_proba_oof = cross_val_predict(fam, X, yF, cv=cv, method="predict_proba", n_jobs=-1)
    gen_proba_oof = cross_val_predict(gen, X, yG, cv=cv, method="predict_proba", n_jobs=-1)

    # Fit final models on full data for inference and save
    fam.fit(X, yF)
    gen.fit(X, yG)
    dump(fam, MODELS / "Stage2_family_stacker.joblib")
    dump(gen, MODELS / "Stage2_genus_stacker.joblib")

    np.save(MODELS / "Stage2_family_oof.npy", fam_proba_oof)
    np.save(MODELS / "Stage2_genus_oof.npy", gen_proba_oof)
    print("[Stage2] Saved family/genus models and OOF probabilities.")
    return fam_proba_oof, gen_proba_oof


In [5]:
# -----------------
# Stage 3: Train final booster on meta features (p_rare_oof + fam/gen OOF)
# -----------------
def stage3_train():
    df, _ = load_xy()
    y_species = df["species"].to_numpy()

    p_rare = np.load(MODELS / "SVM" / "p_rare_oof.npy")
    fam_oof = np.load(MODELS / "Stage2_family_oof.npy")
    gen_oof = np.load(MODELS / "Stage2_genus_oof.npy")

    n = min(len(p_rare), len(fam_oof), len(gen_oof), len(y_species))
    metaX = np.hstack([p_rare[:n].reshape(-1,1), fam_oof[:n], gen_oof[:n]])
    y = y_species[:n]

    Xtr, Xte, ytr, yte = train_test_split(metaX, y, test_size=0.2, stratify=y, random_state=RS)

    ada = AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=3, random_state=RS),
        n_estimators=400, learning_rate=0.8, algorithm="SAMME", random_state=RS
    )
    ada.fit(Xtr, ytr)
    yhat = ada.predict(Xte)
    print("\n[Stage3 – AdaBoost]")
    print(classification_report(yte, yhat, digits=3))
    print("Accuracy:", accuracy_score(yte, yhat))

    dump(ada, MODELS / "Stage3_Booster.joblib")
    print("[Stage3] Saved Stage3_Booster.joblib")
    return ada

In [6]:
# -----------------
# Inference on new data (uses saved Stage1 best SVM + Stage2 models + Stage3 booster)
# -----------------
def load_best_svm_rarity():
    meta = json.load(open(MODELS / "SVM" / "_best.json", "r", encoding="utf-8"))
    model_path = MODELS / "SVM" / f"{meta['tag']}.joblib"
    clf = load(model_path)
    return clf, meta

def run_inference(new_parquet_path):
    df_new = pd.read_parquet(new_parquet_path)
    X_new = df_new.drop(columns=["family","genus","species"], errors="ignore").to_numpy(np.float32)

    svm_gate, meta = load_best_svm_rarity()
    p_rare = svm_gate.predict_proba(X_new)[:, 1]

    fam = load(MODELS / "Stage2_family_stacker.joblib")
    gen = load(MODELS / "Stage2_genus_stacker.joblib")
    fam_proba = fam.predict_proba(X_new)
    gen_proba = gen.predict_proba(X_new)

    booster = load(MODELS / "Stage3_Booster.joblib")
    metaX = np.hstack([p_rare.reshape(-1,1), fam_proba, gen_proba])
    species_pred = booster.predict(metaX)
    conf = booster.predict_proba(metaX).max(axis=1)

    out = df_new.copy()
    out["species_pred"] = species_pred
    out["confidence"] = conf
    return out


In [ ]:
best = stage1_train(train_size=0.75, methods=("gmm","kmeans","knn"))
_ = stage1_make_oof_prare(best)
_ = stage2_train()
_ = stage3_train()